In [ ]:
#@title Installing dependencies:
# installing foundry:
!curl -L https://foundry.paradigm.xyz | bash
!source ~/.bashrc
import os
os.environ['PATH'] += ":/root/.foundry/bin"
!foundryup

# cloning repo and installing foundry dependencies:
!git clone -b main https://github.com/baking-bad/etherlink-bridge.git
%cd etherlink-bridge/
!git submodule update --init --recursive

# installing python dependencies:
!pip install poetry
!poetry install

### Environment Initialization:

Run the following cell to set up the environment and provide private keys for interacting *with* the bridge. Default values are provided for all variables; however, you may wish to deploy your own token and establish a bridge using your own keys.

In [6]:
!poetry run init_wallets

Enter "L1_PRIVATE_KEY" (default: edsk2nu78mRwg4V5Ka7XCJFVbVPPwhry8YPeEHRwzGQHEpGAffDvrH): 
Enter "L1_PUBLIC_KEY_HASH" (default: tz1YG6P2GTQKFpd9jeuESam2vg6aA9HHRkKo): 
Enter "L1_RPC_URL" (default: https://rpc.tzkt.io/oxfordnet/): 
Enter "L1_ROLLUP_ADDRESS" (default: sr19t5DDEhvnPaNZgNYVAkmBRd5nAPyMU7nP): sr1QgYF6ARMSLcWyAX4wFDrWFaZTyy4twbqe
Enter "L2_PRIVATE_KEY" (default: 8636c473b431be57109d4153735315a5cdf36b3841eb2cfa80b75b3dcd2d941a): 
Enter "L2_PUBLIC_KEY" (default: 0xBefD2C6fFC36249ebEbd21d6DF6376ecF3BAc448): 
Enter "L2_MASTER_KEY" (default: 9722f6cc9ff938e63f8ccb74c3daa6b45837e5c5e3835ac08c44c50ab5f39dc0): 
Enter "L2_RPC_URL" (default: https://etherlink.dipdup.net): 
Enter "L2_ROLLUP_RPC_URL" (default: https://etherlink-rollup-oxford.dipdup.net): 
Enter "L2_KERNEL_ADDRESS" (default: 0x0000000000000000000000000000000000000000): 
Enter "L2_WITHDRAW_PRECOMPILE_ADDRESS" (default: 0x0000000000000000000000000000000000000040): 


### Bridge token pair listing:

The **Ticket Transport Layer**, a key component of the bridge, enables ticket transfers between Tezos and Etherlink. To list a new token pair and establish a bridge, users need to:
1. If the token is an **FA2** or **FA1.2** standard token that doesn’t natively support tickets, the user must deploy a **Ticketer** contract on the Tezos side. This contract links to the specific Tezos token to convert it into a ticket.
2. The user then deploys an **ERC20Proxy** on the Etherlink side – a smart contract implementing ERC20. This contract should be configured to expect tickets from the deployed **Ticketer** including ticketer address and ticket content. This setup allows the rollup to mint ERC20 tokens corresponding to the incoming tickets from Tezos.
3. Additionally, deploy a **TicketHelper** on the Tezos side, targeting the specific token and **Ticketer** pair. This step is necessary as the **transfer_ticket** operation is not currently supported by wallets. The **TicketHelper** facilitates the wrapping of tokens into tickets and their transfer to the rollup in a single transaction.

#### Deploying a Token

For demonstration purposes, users can deploy a test token that will later be bridged. The bridge has been tested with two types of tokens, which are available in the repository:
- The **FA1.2** **Ctez** token.
- The **FA2** **fxhash** token.

To deploy this token and allocate the total supply to the token's originator, execute the following script. The example below illustrates the deployment for an **FA2** token type:

In [8]:
!poetry run deploy_token --token-type FA2 --token-id 42

deploying contract from filename /content/etherlink-bridge/etherlink-bridge/etherlink-bridge/tezos/tests/helpers/../tokens/fa2-fxhash.tz
found contract address: KT1GemMPvp2bV8TUV1DzWTwdhT27TtMgJiTw


#### Deploying a Ticketer

To deploy a ticketer for a specific token address and token id, use the following script. The example below uses a **FA1.2** token previously deployed on Oxfordnet:

In [9]:
!poetry run deploy_ticketer --token-address KT1GemMPvp2bV8TUV1DzWTwdhT27TtMgJiTw --token-type FA2 --token-id 42

deploying contract from filename /content/etherlink-bridge/etherlink-bridge/etherlink-bridge/tezos/tests/helpers/../../build/ticketer.tz
found contract address: KT1GQEybCQffb6YJ5NH9GhPEeRyufrYP3amN
address_bytes: 0155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc00
content_bytes: 0707002a05090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a000000160158827df9def8a4d2c92152c872c553b01ff3de0b0007040100000008746f6b656e5f69640a0000000305002a0704010000000a746f6b656e5f747970650a00000009050100000003464132


During the Ticketer's deployment, user will receive its parameters, including **address_bytes** and **content_bytes**. These are required for the origination of the **ERC20Proxy**.

#### Deploying ERC20Proxy

To deploy a token contract on the Etherlink side, which will mint tokens upon deposit, execute the script below. This script requires **ticketer-address-bytes** and **ticketer-content-bytes**. The example uses bytes from a ticketer previously deployed in Nairobinet:

In [12]:
!poetry run deploy_erc20 --ticketer-address-bytes 0155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc00 --ticket-content-bytes 0707002a05090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a000000160158827df9def8a4d2c92152c872c553b01ff3de0b0007040100000008746f6b656e5f69640a0000000305002a0704010000000a746f6b656e5f747970650a00000009050100000003464132 --token-name "FA2 Test Token, id: 42" --token-symbol "FA2[42]" --decimals 0

Successfully deployed ERC20 contract:
Compiling 29 files with 0.8.21
Solc 0.8.21 finished in 9.87s
Compiler run successful!
Deployer: 0xBefD2C6fFC36249ebEbd21d6DF6376ecF3BAc448
Deployed to: 0x87dcBf128677ba36E79D47dAf4eb4e51610e0150
Transaction hash: 0x5d5049670d32a39c8c3582ebc7ca6f83b781f67c50fb297da6c5d72caa0d5456



NOTE: To obtain the **ticketer-address-bytes** and **content-bytes** for an already deployed ticketer, user can use the script below:

In [14]:
!poetry run get_ticketer_params --ticketer KT1GQEybCQffb6YJ5NH9GhPEeRyufrYP3amN

address_bytes: 0155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc00
content_bytes: 0707002a05090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a000000160158827df9def8a4d2c92152c872c553b01ff3de0b0007040100000008746f6b656e5f69640a0000000305002a0704010000000a746f6b656e5f747970650a00000009050100000003464132


#### Deploying a Ticket Helper

To enable Tezos wallets to interact with specific tickets, deploy a Ticket Helper using the script below. The example provided uses a ticketer that was previously deployed in Oxfordnet:

In [10]:
!poetry run deploy_ticket_helper --ticketer-address KT1GQEybCQffb6YJ5NH9GhPEeRyufrYP3amN

deploying contract from filename /content/etherlink-bridge/etherlink-bridge/etherlink-bridge/tezos/tests/helpers/../../build/ticket-helper.tz
found contract address: KT1LstLU529PtDUQHo2x8WybNXBzLXnF6Tkv


### Deposit

To make a deposit, user need to transfer Tickets to the rollup address with attached Routing Info in the [specified format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#deposit): `| receiver | proxy |` 40 bytes payload, both receiver and proxy are standard Ethereum addresses in raw form (H160). A script is available to facilitate this process. It requires the Ticket Helper address as **ticket-helper-address**, the Etherlink ERC20Proxy contract as **proxy-address**, and the bridged amount as the **amount** variable. Here's an example:

In [15]:
!poetry run deposit --ticket-helper-address KT1LstLU529PtDUQHo2x8WybNXBzLXnF6Tkv --proxy-address 0x87dcBf128677ba36E79D47dAf4eb4e51610e0150 --amount 7777

Succeed, transaction hash: opG2vhZ2x2p6PVcQoLq91gVSw3gZERyrDmhRUMAXqUnETGmyZnE


NOTE: This script performs two operations in bulk. The first operation approves the token for the **Ticket Helper**, and the second operation makes a call to the **Ticket Helper** **deposit** entrypoint.

### Withdrawal Process

The withdrawal process involves two key steps:
1. Initiating the withdrawal on the Etherlink side, which results in the creation of an outbox message in the rollup commitment.
2. Completing the withdrawal by executing the outbox message on the Tezos side once the commitment has settled (TODO: Clarify terminology if necessary).

#### Etherlink Withdrawal

To initiate a withdrawal, user need to call the withdrawal precompile on the Etherlink side. This requires providing the **ERC20Proxy** address along with Routing Info in a [specific format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#withdrawal), two forged contracts concatenated: `| receiver | proxy |` 44 bytes. Forged contact consists of binary suffix/prefix and body (blake2b hash digest). A script is available to facilitate the withdrawal process. It requires the ERC20 contract (which will burn tokens) as **proxy-address**, the Tezos router address (which will receive the ticket from the rollup) as **router-address**, and the bridged amount as **amount**. Additionally, the **ticketer-address-bytes** and **ticketer-content-bytes** are required to allow **ERC20Proxy** validate token before burning.

NOTE: For automatic unwrapping of Tezos tickets back into tokens, the **Ticketer** address can be provided as the **router-address**.

In [17]:
!poetry run withdraw --proxy-address 0x87dcBf128677ba36E79D47dAf4eb4e51610e0150 --amount 1080 --ticketer-address-bytes 0155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc00 --ticket-content-bytes 0707002a05090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a000000160158827df9def8a4d2c92152c872c553b01ff3de0b0007040100000008746f6b656e5f69640a0000000305002a0704010000000a746f6b656e5f747970650a00000009050100000003464132 --router-address KT1GQEybCQffb6YJ5NH9GhPEeRyufrYP3amN

Successfully called withdraw:

blockHash               0x856472e5f89691a8e34dd3ddd8882e15dd9b89c80b39e0a275824931e6b0268c
blockNumber             413
contractAddress         
cumulativeGasUsed       29857
effectiveGasPrice       1
from                    0xBefD2C6fFC36249ebEbd21d6DF6376ecF3BAc448
gasUsed                 29857
logs                    [{"address":"0x0000000000000000000000000000000000000000","topics":["0x64e765b73c0829c01c6f3026a840120142d5a0b83626c548cff3a44ca5299c98","0x3dee5129eb8af1183170a7c67f1a7e020eb32f500d761647db62c908a67d44a2"],"data":"0x000000000000000000000000befd2c6ffc36249ebebd21d6df6376ecf3bac44800000000000000000000000087dcbf128677ba36e79d47daf4eb4e51610e015000008a7390072a389159c73687165cd7910e8a39160600000000000000000000000000000000000000000000000000000000000000000000000000000000043800000000000000000000000000000000000000000000000000000000002b3c310000000000000000000000000000000000000000000000000000000000000000","blockHash":"0x856472e5f89691a8e34dd3ddd8882e1

#### Finalizing Tezos Withdrawal

To complete the withdrawal process, user need to call the outbox message on the Tezos side once it has been finalized (settled) on the L1 side. During the withdrawal process, the Etherlink side emits a **Withdrawal** event, which includes **outboxLevel** and **outboxMsgId**.

Users can obtain **outboxLevel** and **outboxMsgId** from the **Withdrawal** event, which is emitted upon the successful formation of an outbox message. Since both parameters are of type `uint256`, to retrieve **outboxMsgId**, users need to take the last 32 bytes of the event. For **outboxLevel**, take another 32 bytes preceding those and convert them into an integer. Below is an example of how this can be done:
- Here is a link to the block explorer with the withdrawal transaction: http://blockscout.dipdup.net/tx/0xbc6bbb184a43c0eb10d026c897475262ae47af370785844cb8b4788791ea1c5c/logs
- Here is **Withdrawal** event data: `0x000000000000000000000000befd2c6ffc36249ebebd21d6df6376ecf3bac44800000000000000000000000087dcbf128677ba36e79d47daf4eb4e51610e015000008a7390072a389159c73687165cd7910e8a39160600000000000000000000000000000000000000000000000000000000000000000000000000000000043800000000000000000000000000000000000000000000000000000000002b3c310000000000000000000000000000000000000000000000000000000000000000`
- Here is **outboxMsgId** is the last 32 bytes: `0000000000000000000000000000000000000000000000000000000000000000`, which is integer `0`
- Here is **outboxLevel** is the 32 bytes preceding those: `00000000000000000000000000000000000000000000000000000000002b3c31`, which is integer `2833457`

To simplify this process, script to parse withdrawal event added:

In [18]:
!poetry run parse_withdrawal_event --tx-hash 0xbc6bbb184a43c0eb10d026c897475262ae47af370785844cb8b4788791ea1c5c

outbox_level: 2833457
outbox_index: 0


Now, when the user knows the index and outbox level where withdrawal transaction was added, it is possible to get **commitment** and **proof**, which are required for `execute_outbox_message` call. To do this users need to perform call to the `global/block/head/helpers/proofs/outbox/{outbox_level}/messages` Rollup RPC Node endpoint. There is a script which performs this call:

NOTE: Please be aware that it may take some time for L2 transactions to settle on L1. The settlement time is approximately 6 minutes on Oxfordnet and 2 weeks on Mainnet.

In [21]:
!poetry run get_proof --level 2833457 --index 0

commitment: src131WuTJN7FKzefkjQztLLtqxtspQ8hkiakEYGkPPHheSd4Gbdqh
proof: 0300025491ef833458c06460879395e3076b57a3054ed9d77735b4ed9e95da1d883c925491ef833458c06460879395e3076b57a3054ed9d77735b4ed9e95da1d883c920005820764757261626c65d03a20ef6c5502cc8f3acd0502d138a587955dc4be101342215aa1af9368bffcae03746167c00800000004536f6d650003c0c2fd443334020528c36a7bb5e79d1489eddc6aff5bf5884fb040c06073495e98820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc004002b3c370133810f76616c69646974795f706572696f64c00400013b0082013181086f7574626f7865730101a4c08da6efd470f511ffc88ebe18f82a04b6c0d3448da54cf34ecec0dcf8aa136af000d000620035c0d6fb29559569e6db0e78723f825b1faa414d78d92f70a3a01f3f0bf074cae712001d000ec0fd87a058c4304bf39e891761282e6de4779f2bc36ff11d595d24e426bdcaf34b0004c0f12d2dbd6f0c3f9aeeb030dd148c008f46360a7d1445b71c635b892334983e73820732383333313931820468656164c00100066c656e677468c0010007323833333435370003810468

Since this outbox message settled on the L1 side, user able to execute it by making `execute_outbox_message` operation. There is a script which allows to do this using provided private keys:

In [22]:
!poetry run execute_outbox_message --commitment src131WuTJN7FKzefkjQztLLtqxtspQ8hkiakEYGkPPHheSd4Gbdqh --proof 0300025491ef833458c06460879395e3076b57a3054ed9d77735b4ed9e95da1d883c925491ef833458c06460879395e3076b57a3054ed9d77735b4ed9e95da1d883c920005820764757261626c65d03a20ef6c5502cc8f3acd0502d138a587955dc4be101342215aa1af9368bffcae03746167c00800000004536f6d650003c0c2fd443334020528c36a7bb5e79d1489eddc6aff5bf5884fb040c06073495e98820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc004002b3c370133810f76616c69646974795f706572696f64c00400013b0082013181086f7574626f7865730101a4c08da6efd470f511ffc88ebe18f82a04b6c0d3448da54cf34ecec0dcf8aa136af000d000620035c0d6fb29559569e6db0e78723f825b1faa414d78d92f70a3a01f3f0bf074cae712001d000ec0fd87a058c4304bf39e891761282e6de4779f2bc36ff11d595d24e426bdcaf34b0004c0f12d2dbd6f0c3f9aeeb030dd148c008f46360a7d1445b71c635b892334983e73820732383333313931820468656164c00100066c656e677468c0010007323833333435370003810468656164c001008208636f6e74656e7473810130c0e9000000e500000000e007070a0000001600008a7390072a389159c73687165cd7910e8a39160607070a000000160155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc0007070707002a05090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a000000160158827df9def8a4d2c92152c872c553b01ff3de0b0007040100000008746f6b656e5f69640a0000000305002a0704010000000a746f6b656e5f747970650a0000000905010000000346413200b8100155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc00000000087769746864726177066c656e677468c00101c0b36dd661223dea6d37f08ed83e77d834f343bed72d6e63a8ce75803c7da01bd2c0dbba6a59432e57f81c49e91133150af18e122db7f1be4f70b6453e50654de508c02d0cc6dc3c83abfe1fbef3c9473df0b353495af428b7928de73f1ca10f3292b80134810d6d6573736167655f6c696d6974c002a401047761736dd0d3f7aaf1483a2fa12f3a72f99f0bcbc9a4cee57e2350cc4e8185ca9bb8f4e39e5491ef833458c06460879395e3076b57a3054ed9d77735b4ed9e95da1d883c92002b3c310000000000e007070a0000001600008a7390072a389159c73687165cd7910e8a39160607070a000000160155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc0007070707002a05090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a000000160158827df9def8a4d2c92152c872c553b01ff3de0b0007040100000008746f6b656e5f69640a0000000305002a0704010000000a746f6b656e5f747970650a0000000905010000000346413200b8100155c34fe2c664715ec4c0a43a01f1e4141fa9b3cc00000000087769746864726177

Succeed, transaction hash: op54jebFAf5NSeK9ZZqniCZUW1LajJeovQ6cjszKWwEGaZNCdTX


### Other scripts:

- Fund Etherkink account provided as `L2_PUBLIC_KEY`:

In [11]:
!poetry run fund_etherlink_account

Successfully funded account:

blockHash               0x78693a8e4c7fe623ff8874034e5171b744b231e253abb8e78541dba40766f0ab
blockNumber             361
contractAddress         
cumulativeGasUsed       21000
effectiveGasPrice       1
from                    0x6ce4d79d4E77402e1ef3417Fdda433aA744C6e1c
gasUsed                 21000
logs                    []
logsBloom               0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
root                    
status                  1
transactionHash         0xc66826cdab10ffaf36b35fe7d4564a